# First part : Best Cities

In [ ]:
import requests
import pandas as pd
import plotly.express as px
import json
import numpy as np

<div align="Justify">

The aim of this project is to help kayaking customers choose their next French destination, based on the weather forecast for the next few days and the accommodation on offer in the destination. To start with, all we have is a list of 35 French cities.

</div>

## GPS coordinates

In [ ]:
cities = ["Mont Saint Michel", "St Malo", "Bayeux", "Le Havre", "Rouen", "Paris", "Amiens", "Lille", "Strasbourg", "Chateau du Haut Koenigsbourg", "Colmar", "Eguisheim", "Besancon",
          "Dijon", "Annecy", "Grenoble", "Lyon", "Gorges du Verdon", "Bormes les Mimosas", "Cassis", "Marseille", "Aix en Provence", "Avignon", "Uzes", "Nimes", "Aigues Mortes",
          "Saintes Maries de la mer", "Collioure", "Carcassonne", "Ariege", "Toulouse", "Montauban", "Biarritz", "Bayonne", "La Rochelle"]

In [ ]:
cities_gps = []

for city in cities:
    r =  requests.get(f"https://nominatim.openstreetmap.org/search?city={city}&format=json")
    if len(r.json())>0:
      latitude = r.json()[0]["lat"]
      longitude = r.json()[0]["lon"]
      coordinates_gps = {"City":city, "lat":latitude, "lon":longitude}
      cities_gps.append(coordinates_gps)
    else:
      print(f"{city} not found in openstreetmap.")

Gorges du Verdon not found in openstreetmap.
Ariege not found in openstreetmap.


In [ ]:
dataset_cities = pd.DataFrame(cities_gps)
dataset_cities.reset_index(inplace=True)
dataset_cities.rename(columns={'index' : 'ID'}, inplace = True)
dataset_cities

,ID,City,lat,lon
0,0,Mont Saint Michel,48.6359541,-1.511459954959514
1,1,St Malo,48.649518,-2.0260409
2,2,Bayeux,49.2764624,-0.7024738
3,3,Le Havre,49.4938975,0.1079732
4,4,Rouen,49.4404591,1.0939658
5,5,Paris,48.8534951,2.3483915
6,6,Amiens,49.8941708,2.2956951
7,7,Lille,50.6365654,3.0635282
8,8,Strasbourg,48.584614,7.7507127
9,9,Chateau du Haut Koenigsbourg,48.2495226,7.3454923


In [ ]:
dataset_cities.dtypes

ID       int64
City    object
lat     object
lon     object
dtype: object

##  Weather by cities

In [ ]:
API_key = '***'

params = {
    "appid" : API_key,
    "units" : "metric",
    "exclude" : "current,minutely,hourly"
}

In [ ]:
# r = requests.get(f'https://api.openweathermap.org/data/3/weather?q=London,uk', params=params).json()

In [ ]:
# for index, row in dataset_cities.iterrows():
#     lat = row['lat']
#     lon = row['lon']
#     r = requests.get(f"https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}", params=params).json()
#     print(r)

In [ ]:
# Get information, make a dataset with it.
# List for data
data = []

for index, row in dataset_cities.iterrows():
    lat = row['lat']
    lon = row['lon']
    city = row['City']
    ID = row['ID']
    r = requests.get(f"https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}", params=params).json()

    for entry in r['list']:
        # Information required
        date = entry['dt_txt'].split(' ')[0]
        temp = entry['main']['temp']
        description = entry['weather'][0]['description']
        clouds = entry['clouds']['all']
        pop = entry['pop']

        # Add to the dictionnary
        data.append({
            'ID': ID,
            'City': city,
            'lat': lat,
            'lon': lon,
            'date': date,
            'temp': temp,
            'description': description,
            'clouds': clouds,
            'pop': pop
        })

# Creation of dataframe
df = pd.DataFrame(data)

# Let's see it
df.head(50)

,ID,City,lat,lon,date,temp,description,clouds,pop
0,0,Mont Saint Michel,48.6359541,-1.511459954959514,2023-07-23,21.21,moderate rain,100,0.98
1,0,Mont Saint Michel,48.6359541,-1.511459954959514,2023-07-23,20.09,moderate rain,100,0.98
2,0,Mont Saint Michel,48.6359541,-1.511459954959514,2023-07-23,18.34,light rain,100,1.00
3,0,Mont Saint Michel,48.6359541,-1.511459954959514,2023-07-24,15.32,light rain,64,1.00
4,0,Mont Saint Michel,48.6359541,-1.511459954959514,2023-07-24,15.56,light rain,65,0.36
5,0,Mont Saint Michel,48.6359541,-1.511459954959514,2023-07-24,15.27,broken clouds,83,0.04
6,0,Mont Saint Michel,48.6359541,-1.511459954959514,2023-07-24,19.06,broken clouds,76,0.07
7,0,Mont Saint Michel,48.6359541,-1.511459954959514,2023-07-24,21.49,light rain,60,0.54
8,0,Mont Saint Michel,48.6359541,-1.511459954959514,2023-07-24,20.86,light rain,23,0.61
9,0,Mont Saint Michel,48.6359541,-1.511459954959514,2023-07-24,18.78,few clouds,19,0.59


In [ ]:
from statistics import mode
# to_datetime
df['date'] = pd.to_datetime(df['date'])

# Group by city and date
df_groupby = df.groupby(['ID', 'City', 'lat', 'lon', 'date'])

# Mean for 'temp', 'pop', and 'clouds', and mode for 'description'
df_mean_day = df_groupby.agg({
    'temp': 'mean',
    'pop': 'mean',
    'clouds': 'mean',
    'description': lambda x: mode(x) if len(x) > 0 else None
}).reset_index()

# Rename columns
df_mean_day.rename(columns={
    'temp': 'mean_temp',
    'pop': 'mean_pop',
    'clouds': 'mean_clouds',
    'description': 'description_day'}, inplace=True)

df_mean_day.head(15)

,ID,City,lat,lon,date,mean_temp,mean_pop,mean_clouds,description_day
0,0,Mont Saint Michel,48.6359541,-1.511459954959514,2023-07-23,19.880000,0.986667,100.000000,moderate rain
1,0,Mont Saint Michel,48.6359541,-1.511459954959514,2023-07-24,17.673750,0.471250,51.750000,light rain
2,0,Mont Saint Michel,48.6359541,-1.511459954959514,2023-07-25,16.580000,0.301250,48.000000,light rain
3,0,Mont Saint Michel,48.6359541,-1.511459954959514,2023-07-26,17.242500,0.281250,93.375000,overcast clouds
4,0,Mont Saint Michel,48.6359541,-1.511459954959514,2023-07-27,19.658750,0.201250,86.625000,broken clouds
5,0,Mont Saint Michel,48.6359541,-1.511459954959514,2023-07-28,18.666000,0.336000,92.200000,light rain
6,1,St Malo,48.649518,-2.0260409,2023-07-23,19.620000,0.890000,96.666667,moderate rain
7,1,St Malo,48.649518,-2.0260409,2023-07-24,17.190000,0.406250,47.125000,light rain
8,1,St Malo,48.649518,-2.0260409,2023-07-25,16.572500,0.216250,51.875000,light rain
9,1,St Malo,48.649518,-2.0260409,2023-07-26,17.553750,0.296250,96.500000,light rain


In [ ]:
df_mean_day.shape

(198, 9)

In [ ]:
df_mean_day.isna().value_counts()

ID     City   lat    lon    date   mean_temp  mean_pop  mean_clouds  description_day
False  False  False  False  False  False      False     False        False              198
dtype: int64

In [ ]:
df_mean_day['date'].unique()

array(['2023-07-23T00:00:00.000000000', '2023-07-24T00:00:00.000000000',
       '2023-07-25T00:00:00.000000000', '2023-07-26T00:00:00.000000000',
       '2023-07-27T00:00:00.000000000', '2023-07-28T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [ ]:
df_mean_day['date'] = df_mean_day['date'].dt.date

In [ ]:
df_mean_day['lat'] = df_mean_day['lat'].astype(float)
df_mean_day['lon'] = df_mean_day['lon'].astype(float)

In [ ]:
fig = px.scatter_mapbox(df_mean_day, lat="lat", lon="lon", color="mean_temp", zoom = 5,
                        mapbox_style="open-street-map", color_continuous_scale = 'Jet',
                       animation_frame = 'date')

fig.update_traces(marker=dict(size=15))
fig.show()

## Best 5 cities

In [ ]:
# Sort DataFrame by date
df_sorted = df_mean_day.sort_values(by='date')

#  Group by city and take the average temperature, cloud cover and precipitation for each city over the period.
grouped_df = df_sorted.groupby('City').agg({
    'ID': 'first',
    'lat': 'first',
    'lon': 'first',
    'mean_temp': 'max',
    'mean_clouds': 'min',
    'mean_pop': 'min'
}).reset_index()

# Apply a weight to each criterion
grouped_df['score'] = 0.6 * grouped_df['mean_temp'] - 0.3 * grouped_df['mean_pop'] - 0.1 * grouped_df['mean_clouds']

# Sort by descending score
grouped_df = grouped_df.sort_values(by='score', ascending=False)

grouped_df

,City,ID,lat,lon,mean_temp,mean_clouds,mean_pop,score
13,Collioure,26,42.525050,3.083155,32.530000,7.666667,0.000000,18.751333
21,Lyon,16,45.757814,4.832011,32.243333,12.375000,0.000000,18.108500
4,Avignon,21,43.949249,4.805901,30.566667,3.000000,0.000000,18.040000
31,Toulouse,28,43.604462,1.444247,31.776667,11.000000,0.002500,17.965250
17,Grenoble,15,45.187560,5.735782,30.483333,3.625000,0.000000,17.927500
22,Marseille,19,43.296174,5.369953,29.556667,0.666667,0.000000,17.667333
10,Carcassonne,27,43.213036,2.349107,29.566667,2.000000,0.006667,17.538000
1,Aix en Provence,20,43.529842,5.447474,29.056667,1.666667,0.000000,17.267333
11,Cassis,18,43.214036,5.539632,28.746667,0.666667,0.000000,17.181333
25,Nimes,23,43.837425,4.360069,29.096667,4.333333,0.000000,17.024667


In [ ]:
# First 5 cities
top_5_cities = grouped_df.head(5)
top_5_cities

,City,ID,lat,lon,mean_temp,mean_clouds,mean_pop,score
13,Collioure,26,42.525050,3.083155,32.530000,7.666667,0.0000,18.751333
21,Lyon,16,45.757814,4.832011,32.243333,12.375000,0.0000,18.108500
4,Avignon,21,43.949249,4.805901,30.566667,3.000000,0.0000,18.040000
31,Toulouse,28,43.604462,1.444247,31.776667,11.000000,0.0025,17.965250
17,Grenoble,15,45.187560,5.735782,30.483333,3.625000,0.0000,17.927500


In [ ]:
fig = px.scatter_mapbox(top_5_cities, lat="lat", lon="lon", color="score", zoom = 5,
                        mapbox_style="open-street-map", color_continuous_scale = 'Jet')
fig.update_traces(marker=dict(size=25))
fig.show()

## Save everything in S3

In [ ]:
pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 106.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.6 MB/s eta 0:00:00


In [ ]:
import boto3

In [ ]:
# credentials aws
aws_access_key_id = '***'
aws_secret_access_key = '***'
bucket_name = 'certification-block1'

In [ ]:
# For all weather data
# Save data in CSV format
csv_weather_all_informations = df.to_csv(index=False)

# Connexion to AWS S3
s3 = boto3.client('s3',
                  aws_access_key_id=aws_access_key_id,
                  aws_secret_access_key=aws_secret_access_key)

file_name = 'weather_all_informations.csv'

# Sending the CSV file to the S3 bucket
s3.put_object(Bucket=bucket_name, Key=file_name, Body= csv_weather_all_informations)

print("The data was saved in CSV format in the S3 bucket.")

The data was saved in CSV format in the S3 bucket.


In [ ]:
# For data scored
# Save data in CSV format
csv_weather_scored = grouped_df.to_csv(index=False)

# Connexion to AWS S3
s3 = boto3.client('s3',
                  aws_access_key_id=aws_access_key_id,
                  aws_secret_access_key=aws_secret_access_key)

file_name = 'weather_scored.csv'

# Sending the CSV file to the S3 bucket
s3.put_object(Bucket=bucket_name, Key=file_name, Body= csv_weather_scored)

print("The data was saved in CSV format in the S3 bucket.")

The data was saved in CSV format in the S3 bucket.


# Second Part : web scrapping --> Booking.com

In [ ]:
!pip install scrapy -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.2/277.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 102.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 10.2 MB/s eta 0:00:00


In [ ]:
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.selector import Selector
import os
import logging

In [ ]:
!mkdir booking_file

In [ ]:
# Scrapy
class BookingSpider(scrapy.Spider):
    name = "booking"
    start_urls = ['https://www.booking.com']
    cities = grouped_df['City'].unique().tolist()
    checkin = "2023-07-23"
    checkout = "2023-07-28"

    def start_requests(self):
        city_url_template = 'https://www.booking.com/searchresults.fr.html?ss={}&checkin={}&checkout={}&label=gen173nr-1BCAEoggI46AdIM1gEaE2IAQGYAQm4ARfIAQzYAQHoAQGIAgGoAgO4AsHWlqIGwAIB0gIkZTU5NjhmNWEtMTk1Yi00OGEyLThmNWYtYzU2ZDNmMDU5MWZm2AIF4AIB&sid=4044a316139dbb224d418fbb7843881c&aid=304142&lang=fr&sb=1&src_elem=sb&src=index&dest_type=city&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure'
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}

        for city in self.cities:
            city_url = city_url_template.format(city, self.checkin, self.checkout)
            yield scrapy.Request(city_url, headers=headers, callback=self.parse_hotels_with_urls, meta={'city': city, 'city_url': city_url})

    def parse_hotels_with_urls(self, response):
        hotels = response.xpath("//h3[@class='a4225678b2']")

        for hotel in hotels:
            hotel_name = hotel.xpath(".//div[contains(@class, 'fcab3ed991 a23c043802')]/text()").get()
            hotel_url = hotel.xpath(".//@href").get()
            yield response.follow(hotel_url, callback=self.parse_hotels_details, meta={'city': response.meta['city'], 'hotel_name': hotel_name, 'hotel_url': hotel_url})

    def parse_hotels_details(self, response):
         yield {
              'City': response.meta['city'],
              'hotel_name': response.meta['hotel_name'],
              'hotel_url': response.meta['hotel_url'],
              'hotel_rating': response.xpath("//div[@class='b5cd09854e d10a6220b4']/text()").get(),
              'hotel_description': Selector(response).css('div.hp_desc_main_content').xpath('string()').get().strip(),
              'hotel_address': response.xpath("//span[contains(@class, 'hp_address_subtitle')]/text()").get(),
         }

filename = "scraping_all.json"

if filename in os.listdir('booking_file/'):
    os.remove('booking_file/' + filename)

process = CrawlerProcess(settings={
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36',
    'LOG_LEVEL': logging.INFO,
    # 'AUTOTHROTTLE_ENABLED': True,
    # 'AUTOTHROTTLE_START_DELAY': 5.0,
    # 'AUTOTHROTTLE_TARGET_CONCURRENCY': 1.0,
    # 'AUTOTHROTTLE_MAX_DELAY': 60.0,
    "FEEDS": {
        'booking_file/' + filename: {"format": "json"},
    }
})

process.crawl(BookingSpider)
process.start()

INFO:scrapy.utils.log:Scrapy 2.9.0 started (bot: scrapybot)
2023-07-23 14:56:56 [scrapy.utils.log] INFO: Scrapy 2.9.0 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.1, Twisted 22.10.0, Python 3.10.6 (main, May 29 2023, 11:10:38) [GCC 11.3.0], pyOpenSSL 23.2.0 (OpenSSL 3.1.1 30 May 2023), cryptography 41.0.2, Platform Linux-5.15.109+-x86_64-with-glibc2.35
2023-07-23 14:56:56 [scrapy.utils.log] INFO: Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.1, Twisted 22.10.0, Python 3.10.6 (main, May 29 2023, 11:10:38) [GCC 11.3.0], pyOpenSSL 23.2.0 (OpenSSL 3.1.1 30 May 2023), cryptography 41.0.2, Platform Linux-5.15.109+-x86_64-with-glibc2.35
INFO:scrapy.crawler:Overridden settings:
{'LOG_LEVEL': 20,
 'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
               '(KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'}
2023-07-23 14:56:56 [scrapy.crawle

In [ ]:
# Loading JSON file results into a DataFrame
df_hotel_all= pd.read_json('booking_file/' + filename)

# DataFrame to manipulate and analyze the extracted data
df_hotel_all.head()

In [ ]:
# For all accomodation data
# Save data in CSV format
accommodation_all_cities = df_hotel_all.to_csv(index=False)

# Connexion to AWS S3
s3 = boto3.client('s3',
                  aws_access_key_id=aws_access_key_id,
                  aws_secret_access_key=aws_secret_access_key)

file_name = 'accommodation_all_cities.csv'

# Sending the CSV file to the S3 bucket
s3.put_object(Bucket=bucket_name, Key=file_name, Body= accommodation_all_cities)

print("The data was saved in CSV format in the S3 bucket.")

# Last part : RDS & Database

In [ ]:
# Install sqlalchemy
!pip install sqlalchemy==2.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 29.3 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.19
    Uninstalling SQLAlchemy-2.0.19:
      Successfully uninstalled SQLAlchemy-2.0.19


In [ ]:
from sqlalchemy import create_engine, Column, MetaData, Table, text, Integer, String
from sqlalchemy.dialects.postgresql import VARCHAR, INTEGER

### Database

In [ ]:
FILE_NAME = 'accommodation_all_cities.csv'
s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)
s3.download_file(bucket_name, FILE_NAME, FILE_NAME)

In [ ]:
## Credential RDS
DBHOST = "***"
DBUSER = "***"
DBPASS = "***"
DBNAME = "postgres"
PORT = "5432"

In [ ]:
# Create a PostgreSQL database connection string with the psycopg2 driver (for PostgreSQL)
connection_string = f'postgresql+psycopg2://{DBUSER}:{DBPASS}@{DBHOST}/{DBNAME}'

# Create a SQLAlchemy database engine
engine = create_engine(connection_string)

# Create a MetaData object to define the table structure
metadata = MetaData()

In [ ]:
# Loading the CSV file into a DataFrame
df_accommodation = pd.read_csv(FILE_NAME)
df_accommodation.dtypes

City                 object
hotel_name           object
hotel_url            object
hotel_rating         object
hotel_description    object
hotel_address        object
dtype: object

In [ ]:
# part 1
# Determine the appropriate data types for each DataFrame column
dtype_mapping = {
    'object': String,
    'int64': Integer,
    'float64': Integer
}

# Create a list of columns with the appropriate data types
columns = [Column(col, dtype_mapping[str(df_accommodation[col].dtype)]) for col in df_accommodation.columns]

# Create a table with the same columns as the DataFrame and the appropriate data types
my_table = Table('hotels', metadata, *columns)

# Create the table in the database (only if it doesn't exist)
metadata.create_all(engine)

In [ ]:
# Create a MetaData object to reflect the structure of the 'hotels' table
hotel_table = Table('hotels', metadata, autoload=True, autoload_with=engine)

# Display the column names of the 'hotels' table
print(hotel_table.columns.keys())

['City', 'hotel_name', 'hotel_url', 'hotel_rating', 'hotel_description', 'hotel_address']


In [ ]:
# part 2
# Inserting DataFrame data into the table
df_accommodation.to_sql('hotels', engine, if_exists='append', index=False)

825

In [ ]:
FILE_NAME = 'weather_scored.csv'
s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)
s3.download_file(bucket_name, FILE_NAME, FILE_NAME)

In [ ]:
# Loading the CSV file into a DataFrame
df_weather = pd.read_csv(FILE_NAME)
df_weather.dtypes

City            object
ID               int64
lat            float64
lon            float64
mean_temp      float64
mean_clouds    float64
mean_pop       float64
score          float64
dtype: object

In [ ]:
df_weather.shape

(33, 8)

In [ ]:
# part 1
# Determine the appropriate data type for each column in the DataFrame
dtype_mapping = {
    'object': String,
    'int64': Integer,
    'float64': Integer
}

# Create a list of columns with the appropriate data types
columns = [Column(col, dtype_mapping[str(df_weather[col].dtype)]) for col in df_weather.columns]

# Create a table with the same columns as the DataFrame and the appropriate data types
my_table = Table('sortedweather', metadata, *columns)

# Create the table in the database (only if it doesn't exist)
metadata.create_all(engine)

In [ ]:
#  Create a MetaData object to reflect the structure of the 'weather' table
weather_table = Table('sortedweather', metadata, autoload=True, autoload_with=engine)

# Display 'weather' table column names
print(weather_table.columns.keys())

['City', 'ID', 'lat', 'lon', 'mean_temp', 'mean_clouds', 'mean_pop', 'score']


In [ ]:
# part 2
# Inserting DataFrame data into the table
df_weather.to_sql('sortedweather', engine, if_exists='append', index=False)

33

### Request

In [ ]:
import psycopg2

In [ ]:
# Create a PostgreSQL database connection string
connection_string = f'host={DBHOST} dbname={DBNAME} user={DBUSER} password={DBPASS}'

# Connexion database
connection = psycopg2 .connect(connection_string)

In [ ]:
# Execute the first query to obtain the names of the 5 cities with the best 'score'.
query_weather = """
SELECT *
FROM sortedweather
ORDER BY score DESC
LIMIT 5;
"""

# Get result of request in a dataframe
df_cities = pd.read_sql_query(query_weather, connection)

# Get city names from the DataFrame
top_5_city_names = df_cities['City'].tolist()
top_5_city_names

<ipython-input-64-a1d1b27e32b7>:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_cities = pd.read_sql_query(query_weather, connection)


['Collioure', 'Avignon', 'Toulouse', 'Grenoble', 'Lyon']

In [ ]:
# Execute the second query to obtain the 20 hotels with the highest 'hotel_rating' for the 5 selected cities.
query_hotels = f"""
SELECT hotels.*, sortedweather.mean_temp, sortedweather.mean_clouds, sortedweather.mean_pop, sortedweather.score
FROM hotels
JOIN sortedweather ON hotels."City" = sortedweather."City"
WHERE hotels."City" IN ({','.join(['%s'] * len(top_5_city_names))})
AND hotels."hotel_rating" IS NOT NULL
ORDER BY hotels."hotel_rating" DESC
LIMIT 20;
"""
# Get result of request in a dataframe
df_results = pd.read_sql_query(query_hotels, connection, params=top_5_city_names)

# print(df_results)

<ipython-input-113-bac2917c8e07>:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_results = pd.read_sql_query(query_hotels, connection, params=top_5_city_names)


In [ ]:
df_results.head()

,City,hotel_name,hotel_url,hotel_rating,hotel_description,hotel_address,mean_temp,mean_clouds,mean_pop,score
0,Grenoble,Le petit Versailles - T2 au centre-ville,https://www.booking.com/hotel/fr/le-petit-vers...,"9,8",Vous pouvez bénéficier d'une réduction Genius ...,"\n22 Rue Joseph Rey, 38000 Grenoble, France\n",30,4,0,18
1,Avignon,Magnifique studio,https://www.booking.com/hotel/fr/magnifique-st...,"9,5","Le Magnifique studio est situé à Avignon, à 3,...","\n4 Impasse des Violettes, 84000 Avignon, Fran...",31,3,0,18
2,Collioure,Intense Boutik-Hôtel,https://www.booking.com/hotel/fr/intense-bouti...,"9,4","Situé à Port-Vendres, à 1,4 km de la plage d'O...","\n21 Avenue castellane, 66660 Port-Vendres, Fr...",33,8,0,19
3,Toulouse,expat renting - Entre Piment & Violette - Sain...,https://www.booking.com/hotel/fr/expat-renting...,"9,3",L'Expat renting - Entre Piment & Violette - Sa...,"\n17 Rue Pasteur, 31400 Toulouse, France\n",32,11,0,18
4,Toulouse,logement entier spacieux/lumineux/parking privé,https://www.booking.com/hotel/fr/logement-enti...,9.2,Le Logement entier spacieux/lumineux/parking p...,"\n19 Boulevard Pierre et Marie Curie, 31200 To...",32,11,0,18


In [ ]:
df_results['City'].value_counts()

Toulouse     6
Grenoble     4
Avignon      4
Collioure    4
Lyon         2
Name: City, dtype: int64

### Map

In [ ]:
df_results.columns

Index(['City', 'hotel_name', 'hotel_url', 'hotel_rating', 'hotel_description',
       'hotel_address', 'mean_temp', 'mean_clouds', 'mean_pop', 'score'],
      dtype='object')

In [ ]:
df_results['hotel_address'] = df_results['hotel_address'].str.strip()

In [ ]:
!pip install geopy ipyleaflet -q

In [ ]:
from geopy.geocoders import Nominatim
from ipyleaflet import Map, Marker, Popup, basemaps, FullScreenControl
from ipywidgets import HTML

In [ ]:
geolocator = Nominatim(user_agent="my_map_app")

def create_map_with_markers(df):
    # Creation of the map
    my_map = Map(center=(0, 0), zoom=12, basemap=basemaps.OpenStreetMap.Mapnik)

    # For each adress in the df
    for address in df['hotel_address']:
        location = geolocator.geocode(address)
        if location:
            # Create a widget DOM
            address_widget = HTML(value=address)

            # Add marker
            marker = Marker(location=(location.latitude, location.longitude))
            popup = Popup(location=(location.latitude, location.longitude), child=address_widget)
            marker.popup = popup
            my_map.add_layer(marker)

    return my_map

In [ ]:
my_map = create_map_with_markers(df_results)

In [ ]:
my_map.add_control(FullScreenControl())
my_map

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

# Conclusion

<div align="Jutify">

Now based on an SQL database, the Kayak team can also analyze the weather and available hotels for each destination by querying them directly.

By taking the top-rated hotels for the 5 best cities based on the weather for the next 5 days, we can see on this map the hotels to suggest to Kayak customers. There's something for every preference, from beach vacations to the more urban Toulouse, but also the mountains accessible from Grenoble or Collioure.

We could also offer Kayak the possibility of coupling this information with data from sites that highlight activities in each area, such as trip advisor, and propose a more complete offer to customers.


</div>